In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.sparse import csr_matrix

# transforming given file into csr_matrix
#from google.colab import files
#uploaded = files.upload() 

with open('train.dat') as f:
    lines = f.readlines()
    
rows = len(lines)

# counting number of term ids which are present in training data
termids = 0
for i in range(rows):
    total = lines[i].split()
    termids += len(total)//2
    
values = np.zeros(termids, dtype=np.float)
indices = np.zeros(termids, dtype=np.int)
ptr = np.zeros(rows+1, dtype=np.long)
n = 0 
cols = 0
for i in range(rows):
    line = lines[i].split()
    for j in range(0, len(line), 2): 
        indices[n] = int(line[j]) - 1
        values[n] = float(line[j+1])
        n += 1
        col = int(line[j]) - 1
        if col+1 > cols:
            cols = col+1
            
    ptr[i+1] = n 
    
cm = csr_matrix((values,indices,ptr), shape=(rows,cols),dtype=np.float)
cm.toarray().shape

ValueError: array is too big; `arr.size * arr.dtype.itemsize` is larger than the maximum possible size.

In [2]:
def bisecting_kmeans(matrix, k, numberOfIterations):
    
    clusters = list()
    
    initialcluster = list()
    for i in range(matrix.shape[0]):
        initialcluster.append(i)
    
    clusters.append(initialcluster)
    
    while len(clusters) < k:

        dropClusterIndex = calculateSSE(matrix, clusters)
        droppedCluster = clusters[dropClusterIndex]
        
        clusterA, clusterB = kmeans(matrix[droppedCluster,:], numberOfIterations)
        del clusters[dropClusterIndex]
        
        actualClusterA = list()
        actualClusterB = list()
        for index in clusterA:
            actualClusterA.append(droppedCluster[index])
            
        for index in clusterB:
            actualClusterB.append(droppedCluster[index])
        
        clusters.append(actualClusterA)
        clusters.append(actualClusterB)
    
    labels = [0] * matrix.shape[0]

    for index, cluster in enumerate(clusters):
        for idx in cluster:
            labels[idx] = index + 1
    return labels

In [3]:
def kmeans(mat,matrix, index_list, k):
    
    init_centroids_index = index_list[:2]
    centroids = mat[[init_centroids_index[0],init_centroids_index[1]],:]
    for itr in range(25):
        print("Iteration " + str(itr) + "\n")
        idx = findCluster(matrix,centroids)
        centroids = findCentroids(matrix,idx)
    index_list1 = []
    index_list2 = []
    for i in range(len(idx)):
        if idx[i] == 1:
            index_list1.append(index_list[i])
        elif idx[i] == 2:
            index_list2.append(index_list[i])
    cluster1 = mat[index_list1,:]
    cluster2 = mat[index_list2,:]
    return index_list1, index_list2, cluster1, cluster2, centroids[0], centroids[1]

In [4]:
def findCentroids(mat, idx, k=2):
    centroids = list()
    for i in range(1,k+1):
        indi = [j for j, x in enumerate(idx) if x == i]
        members = mat[indi,:]
        if (members.shape[0] > 1):
            centroids.append(members.toarray().mean(0))
    
    centroids_csr = csr_matrix(centroids)
    return centroids_csr


In [5]:
def initialCentroids(matrix):
    matrixShuffled = shuffle(matrix, random_state=0)
    return matrixShuffled[:2,:]

In [6]:
#finding the appropriate clusters
def findCluster(mat, centroids):
    idx = list()
    similarityMatrix = mat.dot(centroids.T)

    for i in range(similarityMatrix.shape[0]):
        row = similarityMatrix.getrow(i).toarray()[0].ravel()
        top_indices = row.argsort()[-1]
        idx.append(top_indices + 1)
    return idx

In [7]:
from collections import defaultdict

def csr_idf(matrix, copy=False, **kargs):
    r""" Scale a CSR matrix by idf. 
    Returns scaling factors as dict. If copy is True, 
    returns scaled matrix and scaling factors.
    """
    if copy is True:
        matrix = matrix.copy()
    nrows = matrix.shape[0]
    nnz = matrix.nnz
    ind, val, ptr = matrix.indices, matrix.data, matrix.indptr
    # document frequency
    df = defaultdict(int)
    for i in ind:
        df[i] += 1
    # inverse document frequency
    for k,v in df.items():
        df[k] = np.log(nrows / float(v))  ## df turns to idf - reusing memory
    # scale by idf
    for i in range(0, nnz):
        val[i] *= df[ind[i]]
        
    return df if copy is False else matrix

In [8]:
def csr_l2normalize(matrix, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        matrix = matrix.copy()
    nrows = matrix.shape[0]
    nnz = matrix.nnz
    ind, val, ptr = matrix.indices, matrix.data, matrix.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = float(1.0/np.sqrt(rsum))
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return matrix

In [9]:
def similarity(matrix, centroids):
    similarities = matrix.dot(centroids.T)
    return similarities

In [10]:
#Read CSR matrix from the input file
#csrMatrix = csr_read('train.dat', ftype="csr", nidx=1)

#Scale the CSR matrix by idf (Inverse Document Frequency)
csrIDF = csr_idf(cm, copy=True)

#Normalize the rows of a CSR matrix by their L-2 norm.
csrL2Normalized = csr_l2normalize(csrIDF, copy=True)

#Obtain a dense ndarray representation of the CSR matrix.
denseMatrix = csrL2Normalized.toarray()


In [11]:
def calculateSSE(matrix, clusters):
    
    SSE_list = list()
    SSE_array = []
    
    for cluster in clusters:
        members = matrix[cluster,:]
        SSE = np.sum(np.square(members - np.mean(members)))
        SSE_list.append(SSE)
        
    SSE_array = np.asarray(SSE_list)
    dropClusterIndex = np.argsort(SSE_array)[-1]
            
    return dropClusterIndex

In [ ]:
kValues = list()
scores = list()

for k in range(3, 22, 2):
    labels = bisecting_kmeans(denseMatrix, k, 10)
    
    if (k == 7):
        # write result to output file
        outputFile = open("output.dat", "w")
        for index in labels:
            outputFile.write(str(index) +'\n')
        outputFile.close()

    score = calinski_harabaz_score(denseMatrix, labels)
    kValues.append(k)
    scores.append(score)
    
    print ("For K= %d Calinski Harabaz Score is %f" %(k, score))